In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# 출력 예쁘게 하기
from rich.console import Console
from rich.table import Table

console = Console()

def rich_docs(docs, max_len=140, title="Retriever Results"):
    table = Table(title=title)
    table.add_column("#", justify="right")
    table.add_column("Source")
    table.add_column("Page", justify="right")
    table.add_column("Preview")

    for i, d in enumerate(docs, 1):
        m = d.metadata or {}
        src = (m.get("source","") or "").split("/")[-1]
        page = str(m.get("page_label", m.get("page",0)+1))
        text = (d.page_content or "").strip().replace("\n", " ")
        table.add_row(str(i), src, page, (text[:max_len] + ("…" if len(text) > max_len else "")))

    console.print(table)

In [3]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma

In [5]:
from langchain_community.document_loaders import PyPDFLoader
file_path = "../00_data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf"

print("문서로드중")
loader = PyPDFLoader(file_path)
docs = loader.load()
docs[:5]

문서로드중


[Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2025-07-10T16:11:16+09:00', 'moddate': '2025-09-04T16:51:11+09:00', 'trapped': '/False', 'source': '../00_data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'total_pages': 87, 'page': 0, 'page_label': '1'}, page_content='삼성전자 지속가능경영보고서 2025\nA Journey  Towards \n a Sustainable Future\nA Journey  Towards\n a Sustainable Future'),
 Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2025-07-10T16:11:16+09:00', 'moddate': '2025-09-04T16:51:11+09:00', 'trapped': '/False', 'source': '../00_data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'total_pages': 87, 'page': 1, 'page_label': '2'}, page_content='삼성전자 지속가능경영보고서 2025 02AppendixFacts & Figures PrinciplePlanet PeopleOur Company삼성전자 지속가능경영보고서 2025 02\nA Journey  Towards \n a Sustainable Future\nA Journey  Towards \n a Sustainable F

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)
chunk = splitter.split_documents(docs)
len(chunk)

237

In [7]:
chunk[5:7] 

[Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2025-07-10T16:11:16+09:00', 'moddate': '2025-09-04T16:51:11+09:00', 'trapped': '/False', 'source': '../00_data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'total_pages': 87, 'page': 3, 'page_label': '4'}, page_content="사회공헌 분야에서는 2024년 '삼성 청년SW·AI아카데미'에 375억 원을 \n지원해 2,200 명의 청년 소 프트웨어 인재를 양성했으며, 2025년에는 \n마이스터고 졸업생에게까지 교육기회를 확대할 예정입니다. \n또한 자립 준비 청년들의 주 거안정과 미래준비를 지원하는 '삼성 \n희망디딤돌' 활동은 2024년에 충북과 대전센터를 신규로 개소하여 총 \n14,362명을 지원하였으며, 2025년에는 인천센터를 추가로 설립하여 더 \n많은 청년들을 지원할 예정입니다.\n지난해에 이 어 올해 지속가능경영보고서도 글로벌 공시 규 제 \n프레임워크에 맞춰 발 행됐 습니다. 2024년 한 해 동안 회사의  \n지속가능경영 관리체계, 전략, 이 행 활동, 성과 등을 충실 히 담았으며, \n지속가능경영 웹사이트와도 더욱 긴밀히 연계되도록 구성했습니다.\n삼성전자는 불확실성의 시대에 재도 약의 기반을 다지기 위해 지속 \n노력하고 있습니다. '인재와 기술을 바탕으로 최고의 제품과 서비스를 \n창출하여 인류사회에 공헌한다'는 경영철학에 입각해 기술 리더십으로 \n재도약의 기반을 다지고 새로운 영 역에서 미래 성장동력을 확보해 \n나가겠습니다.\n삼성전자는 이해관계자 여러분의 소중한 의견에 늘 귀를 기울이고 있으며, \n앞으로도 지속가능한 성장 기반을 마련

In [9]:
# 벡터 저장소 만들기
embedding = OpenAIEmbeddings(model = "text-embedding-3-small")
persist_directory = "../07_vectorstore/samsung_2025_db"
collection_name = "samsung2025"

In [ ]:
# 처음 실행시 벡터 저장소 만들기     # 저장소를 만들었다면 또 만들면 안
# vectorstore = Chroma.from_documents(
#     documents = chunk,
#     collection_name = collection_name,
#     persist_directory = persist_directory,
#     embedding = embedding
# )

In [11]:
# 벡터 저장소 불러오기
load_vectorstore = Chroma(
    persist_directory = persist_directory,
    collection_name = collection_name,
    embedding_function = embedding
)

### 1. 키워드기반 + 기본 검색기 = 하이브리드(ensemble)

In [12]:
# uv add rank-bm25

In [13]:
ret_similarity = load_vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k" : 5}
)

In [14]:
# 번외: 메타데이터가 궁금하다
db_docs = load_vectorstore._collection.get(include=["documents", "metadatas"])

In [18]:
db_docs

{'ids': ['2593b008-c3a5-4081-ab16-bb9be87f929c',
  '7336a913-1982-4e56-98a9-2b6ba6fc5cd9',
  '2fc401a7-0977-41f3-8260-4c83d4a01adf',
  '5b375aed-43cc-410e-8d25-a9b1f485f5bc',
  'fd8172f6-eea8-409a-b440-f2edfb008343',
  'e893e6d2-b883-4d80-a6d5-06b4c62508ca',
  'fecd6b91-6294-421a-b59b-844c3207beb7',
  'aa0957b6-ac65-4bd3-8588-552e859d75e6',
  'dec7b756-1b16-424d-bf73-360c1f6ac1f1',
  'afe1ffe4-84d2-4b3e-859e-9d1607f89561',
  '95db32e3-0cbe-4cbd-af26-77f8572ddd6d',
  'bb570aa6-b675-41b7-b76c-af06c78fc3ff',
  '61b1cbc9-1ed7-40dd-b045-d1ff06193cda',
  'eff4bbd8-3dc8-4423-a72b-bde26b46ca43',
  'f54a7bf8-8651-4af0-8618-3603ea27c30b',
  '5ada0b95-a3da-4e8e-b562-e7754045b780',
  '892ad7d4-6beb-4ff7-9ee4-429f5b297449',
  'f83e08c1-16cd-499b-8c44-c7f66a38bdf9',
  '6a8786ee-6d2b-4175-9fa3-e241adc6621a',
  '4e90f446-9770-42aa-a1e4-fdeb3694100d',
  '1c7c9f55-1277-464d-bdb1-3097774fc7cf',
  '5c9c08ab-c023-4091-9b94-e4db8109cfe6',
  '7938b323-e4b1-47f4-ae15-42bf6b7623dc',
  'bf02bbc0-f677-418f-80c3-

In [16]:
db_docs["documents"][:5]

['삼성전자 지속가능경영보고서 2025\nA Journey  Towards \n a Sustainable Future\nA Journey  Towards\n a Sustainable Future',
 '삼성전자 지속가능경영보고서 2025 02AppendixFacts & Figures PrinciplePlanet PeopleOur Company삼성전자 지속가능경영보고서 2025 02\nA Journey  Towards \n a Sustainable Future\nA Journey  Towards \n a Sustainable Future\n삼성전자 지속가능경영보고서 2025\nCEO 메시지\n회사소개\n기업 지배구조\n중대성 평가\n이해관계자 소통\n준법과 윤리경영\n[DX부문] \n추진체계와 주요성과 \n기후변화 \n자원순환\n수자원\n오염물질\n경제성과\n사회성과\n환경성과 \n사업부문별 환경성과 \n[DS부문] \n추진체계와 주요성과 \n기후변화 \n자원순환\n수자원\n오염물질\n독립된 인증인의 인증보고서\nScope 1, 2 온실가스 배출량 검증 의견서 \nScope 3 온실가스 배출량 검증 의견서 \nGRI Index\nTCFD 대조표\nSASB 대조표\nAbout This Report \n임직원\n공급망\n사회공헌\n개인정보보호와 보안\n제품 품질과 안전\nOur Company\nPrinciple\nPlanet\nFacts & Figures Appendix \nPeople\n04\n05\n06\n07\n09\n59\n11\n12\n16\n18\n20\n62\n63\n68\n72\n21\n22\n27\n29\n32\n76\n77\n78\n80\n82\n84\n86\n35\n45\n51\n53\n55',
 '삼성전자 지속가능경영보고서 2025\n03\nOur Company AppendixFacts & Figures PrinciplePlanet People\nOur Company\nCEO 메시지\n회사소개\n기업 지배구조\n중대성 평가\n이해관계자 소통\n

In [17]:
db_docs["metadatas"][:5]

[{'moddate': '2025-09-04T16:51:11+09:00',
  'page': 0,
  'producer': 'Adobe PDF Library 15.0',
  'total_pages': 87,
  'source': '../00_data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf',
  'creationdate': '2025-07-10T16:11:16+09:00',
  'page_label': '1',
  'trapped': '/False',
  'creator': 'Adobe InDesign 15.1 (Macintosh)'},
 {'total_pages': 87,
  'trapped': '/False',
  'page': 1,
  'page_label': '2',
  'producer': 'Adobe PDF Library 15.0',
  'creator': 'Adobe InDesign 15.1 (Macintosh)',
  'creationdate': '2025-07-10T16:11:16+09:00',
  'source': '../00_data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf',
  'moddate': '2025-09-04T16:51:11+09:00'},
 {'creationdate': '2025-07-10T16:11:16+09:00',
  'producer': 'Adobe PDF Library 15.0',
  'total_pages': 87,
  'source': '../00_data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf',
  'moddate': '2025-09-04T16:51:11+09:00',
  'page_label': '3',
  'creator': 'Adobe InDesign 15.1 (Macintosh)',
  'trapped': '/False',
  '

In [19]:
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

In [22]:
from langchain_core.documents import Document
bm_doc = [] # 사실은 chunk를 쓰면 되지만
# 우리는 vectorstore 를 처음에 만들고 더이상 똑같은걸 다시 만들지 않을 것이므로
# load_vectorstore._collection 으로 가져와서 chunk 같은 Document 리스트를 만들어야 함

for content, meta in zip(db_docs["documents"], db_docs["metadatas"]):
    bm_doc.append(Document(page_content=content, metadata=meta))
bm_doc[:3]

[Document(metadata={'moddate': '2025-09-04T16:51:11+09:00', 'page': 0, 'producer': 'Adobe PDF Library 15.0', 'total_pages': 87, 'source': '../00_data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'creationdate': '2025-07-10T16:11:16+09:00', 'page_label': '1', 'trapped': '/False', 'creator': 'Adobe InDesign 15.1 (Macintosh)'}, page_content='삼성전자 지속가능경영보고서 2025\nA Journey  Towards \n a Sustainable Future\nA Journey  Towards\n a Sustainable Future'),
 Document(metadata={'total_pages': 87, 'trapped': '/False', 'page': 1, 'page_label': '2', 'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2025-07-10T16:11:16+09:00', 'source': '../00_data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'moddate': '2025-09-04T16:51:11+09:00'}, page_content='삼성전자 지속가능경영보고서 2025 02AppendixFacts & Figures PrinciplePlanet PeopleOur Company삼성전자 지속가능경영보고서 2025 02\nA Journey  Towards \n a Sustainable Future\nA Journey  Towards \n a Sustainable F

In [23]:
bm25 = BM25Retriever.from_documents(bm_doc) # 키워드 기반 검색기
bm25.k = 5 # 상위 5개 지정

In [ ]:
# 벡터 검색기 + bm25 = 하이브리드 검색기 완성
ret_hybrid = EnsembleRetriever(
    retrievers = [ret_similarity, bm25],
    weights = [0.5, 0.5]
)

In [26]:
question = "삼성 전자의 2025년 전망은?"
result = ret_hybrid.invoke(question)
rich_docs(result, title = "삼성 전자 전망")

                                                  삼성 전자 전망                                                   
┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃  # ┃ Source                                           ┃ Page ┃ Preview                                          ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  1 │ Samsung_Electronics_Sustainability_Report_2025_… │    1 │ 삼성전자 지속가능경영보고서 2025 A Journey       │
│    │                                                  │      │ Towards   a Sustainable Future A Journey         │
│    │                                                  │      │ Towards  a Sustainable Future                    │
│  2 │ Samsung_Electronics_Sustainability_Report_2025_… │    4 │ 삼성전자 지속가능경영보고서 2025 04 Our Company  │
│    │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People   │
│    │                                                  │      │ 주주, 고객, 협력회사, 그리고 임직원 여러분,      │
│    │                                                  │      │ 2024년은 글로벌 지정학적 리스크와 AI 기술의 성 … │
│    │                                                  │      │ …                                                │
│  3 │ Samsung_Electronics_Sustainability_Report_2025_… │   76 │ 삼성전자 지속가능경영보고서 2025 76 독립된       │
│    │                                                  │      │ 인증인의 인증보고서 Our Company AppendixFacts &  │
│    │                                                  │      │ Figures PrinciplePlanet People                   │
│  4 │ Samsung_Electronics_Sustainability_Report_2025_… │   77 │ 삼성전자 지속가능경영보고서 2025 77 Scope 1, 2   │
│    │                                                  │      │ 온실가스 배출량 검증 의견서 Our Company          │
│    │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People   │
│  5 │ Samsung_Electronics_Sustainability_Report_2025_… │   86 │ 삼성전자 지속가능경영보고서 2025 86              │
│    │                                                  │      │ 삼성전자주식회사는 경제·사회·환경적 가치 창출    │
│    │                                                  │      │ 성과를 다양한 이해관계자와 투명하게 소통하기     │
│    │                                                  │      │ 위해 2025년 열여덟 번째 지속가능경영보고서를     │
│    │                                                  │      │ 발간합니다. 작성 기준 본 보고서는 지속가능경영   │
│    │                                                  │      │ 보고 기준인 GRI(G…                               │
│  6 │ Samsung_Electronics_Sustainability_Report_2025_… │   52 │ 삼성전자 지속가능경영보고서 2025 52 삼성         │
│    │                                                  │      │ 드림클래스 삼성 드림클래스는 임직원의 후원과     │
│    │                                                  │      │ 참여를 바탕으로 교육 여건이 부족한  국내         │
│    │                                                  │      │ 중학생들이 꿈을 찾고 미래를 준비할 수 있도록     │
│    │                                                  │      │ 멘토링과 맞춤형  진로 교육을 제공하는 프         │
│    │                                                  │      │ 로그램으로 대학생 멘 토, …                       │
│  7 │ Samsung_Electronics_Sustainability_Report_2025_… │   63 │ 삼성전자 지속가능경영보고서 2025 63 사회성과     │
│    │                                                  │      │ 준법·윤리경영 2022년 2023년 2024년 [컴플라이언 … │
│    │                                                  │      │ 교육] 컴플라이언스 교육 1) 명 126,867  138,742   │
│    │                                                  │      │ 138,414 [부정 예방 교육] 부정 예방 교육 2) 명    │
│    │                                                  │      │ 254,045  …                                       │
│  8 │ Samsung_Electronics_Sustainability_Report_2025_… │   51 │ 프로그램을 운영하고 있습니다. 추진 방향          │
│    │                                                  │      │ 삼성전자는 ‘함께가요 미래로! Enabling P eople’   │
│    │                                                  │      │ 비전 아래 , 교육의  기회에서 소외되는 학생 없이  │
│    

### 2-1. 압축 검색기 (Compression retriever)
- 검색 된 문서가 길 때 -> LLM 을 이용해서 내용을 압축해보기
- 문서 내용이 너무 파편화 되어 있는 경우 -> 압축 진행 -> 찌꺼기가 제거 (띄어쓰기, 이모티콘)
- 비용문제 -> 각 문서별로 전부 압축을 진행해서 사용하기 때문

In [27]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor, LLMChainFilter
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model = "gpt-4.1-mini",
    temperature = 0
)

In [28]:
# 압축기 생성
compressor = LLMChainExtractor.from_llm(model)

# 압축 검색기 생성 = 유사도 검색 -> 문서 내용 압축
com_retriever = ContextualCompressionRetriever(
    base_retriever = ret_similarity,
    base_compressor = compressor
)

In [29]:
question = "삼성 전자의 목표와 기준 년도만 간단히 알려줘"
com_result = com_retriever.invoke(question)
com_result

[Document(metadata={'source': '../00_data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'page': 3, 'total_pages': 87, 'trapped': '/False', 'page_label': '4', 'creationdate': '2025-07-10T16:11:16+09:00', 'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'moddate': '2025-09-04T16:51:11+09:00'}, page_content="삼성전자는 2022년 9월 발표한 '新환경경영전략'을 기반으로 탄소중립 달성, 자원순환 극대화, 그리고 기술 혁신을 통한 환경 난제 해결을 위해 노력하고 있습니다.  \nDX(Device eXperience)부문은 2030년 탄소중립 달성을 목표로  \nDS(Device Solutions)부문은 2050년 탄소중립 달성을 목표로"),
 Document(metadata={'total_pages': 87, 'creationdate': '2025-07-10T16:11:16+09:00', 'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'moddate': '2025-09-04T16:51:11+09:00', 'source': '../00_data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'trapped': '/False', 'page': 0, 'page_label': '1'}, page_content='삼성전자 지속가능경영보고서 2025'),
 Document(metadata={'trapped': '/False', 'page_label': '86', 'page': 85, 'total_pag

In [30]:
rich_docs(com_result, title = "압축기 검색기 사용")

                                                압축기 검색기 사용                                                 
┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ # ┃ Source                                           ┃ Page ┃ Preview                                           ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1 │ Samsung_Electronics_Sustainability_Report_2025_… │    4 │ 삼성전자는 2022년 9월 발표한 '新환경경영전략'을   │
│   │                                                  │      │ 기반으로 탄소중립 달성, 자원순환 극대화, 그리고   │
│   │                                                  │      │ 기술 혁신을 통한 환경 난제 해결을 위해 노력하고   │
│   │                                                  │      │ 있습니다.   DX(Device eXperience)부문은 2030년    │
│   │                                                  │      │ 탄소중립 달성을 목표로   …                        │
│ 2 │ Samsung_Electronics_Sustainability_Report_2025_… │    1 │ 삼성전자 지속가능경영보고서 2025                  │
│ 3 │ Samsung_Electronics_Sustainability_Report_2025_… │   86 │ 삼성전자 지속가능경영보고서 2025                  │
│   │                                                  │      │ 삼성전자주식회사는 경제·사회·환경적 가치 창출     │
│   │                                                  │      │ 성과를 다양한 이해관계자와 투명하게 소통하기 위 … │
│   │                                                  │      │ 2025년 열여덟 번째 지속가능경영보고서를           │
│   │                                                  │      │ 발간합니다.   보고 기간   2024년 1월1일부터       │
│   │                                                  │      │ 2024년 12월 …                                     │
└───┴──────────────────────────────────────────────────┴──────┴───────────────────────────────────────────────────┘

In [31]:
for doc in com_result:
    print(doc.page_content, "\n", "-"*60)

삼성전자는 2022년 9월 발표한 '新환경경영전략'을 기반으로 탄소중립 달성, 자원순환 극대화, 그리고 기술 혁신을 통한 환경 난제 해결을 위해 노력하고 있습니다.  
DX(Device eXperience)부문은 2030년 탄소중립 달성을 목표로  
DS(Device Solutions)부문은 2050년 탄소중립 달성을 목표로 
 ------------------------------------------------------------
삼성전자 지속가능경영보고서 2025 
 ------------------------------------------------------------
삼성전자 지속가능경영보고서 2025  
삼성전자주식회사는 경제·사회·환경적 가치 창출 성과를 다양한 이해관계자와 투명하게 소통하기 위해 2025년 열여덟 번째 지속가능경영보고서를 발간합니다.  
보고 기간  
2024년 1월1일부터 2024년 12월 31일까지의 경제·사회·환경적 성과와 활동을 담고 있으며, 일부 성과에 대해서는 2025년 6월 까지의 정보를 포함하고 있습니다. 
 ------------------------------------------------------------


### 2-2. 임베딩 기반 경량 압축(비용X)

In [ ]:
from langchain.retrievers.document_compressors import EmbeddingsFilter

# ret_mmr = load_vectorstore.as_retriever(   # 보여주기용으로 작성함 안써도됌
#     search_type = "mmr",
#     search_kwargs = {"k" : 5,
#                      "fetch_k" : 20,
#                      "lambda_mult" : 0.5
#                      }
# )

emb_filter = EmbeddingsFilter(
    embeddings=embedding,
    similarity_threshold=0.2
)
comp_embed = ContextualCompressionRetriever(
    base_retriever=ret_similarity,
    base_compressor=emb_filter
)
comp_embed_result = comp_embed.invoke(question)

In [34]:
rich_docs(comp_embed_result, title = "임베딩 기반 필터기법")

                                               임베딩 기반 필터기법                                                
┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ # ┃ Source                                           ┃ Page ┃ Preview                                           ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1 │ Samsung_Electronics_Sustainability_Report_2025_… │    4 │ 삼성전자 지속가능경영보고서 2025 04 Our Company   │
│   │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People    │
│   │                                                  │      │ 주주, 고객, 협력회사, 그리고 임직원 여러분,       │
│   │                                                  │      │ 2024년은 글로벌 지정학적 리스크와 AI 기술의 성장  │
│   │                                                  │      │ …                                                 │
│ 2 │ Samsung_Electronics_Sustainability_Report_2025_… │    1 │ 삼성전자 지속가능경영보고서 2025 A Journey        │
│   │                                                  │      │ Towards   a Sustainable Future A Journey  Towards │
│   │                                                  │      │ a Sustainable Future                              │
│ 3 │ Samsung_Electronics_Sustainability_Report_2025_… │   86 │ 삼성전자 지속가능경영보고서 2025 86               │
│   │                                                  │      │ 삼성전자주식회사는 경제·사회·환경적 가치 창출     │
│   │                                                  │      │ 성과를 다양한 이해관계자와 투명하게 소통하기 위 … │
│   │                                                  │      │ 2025년 열여덟 번째 지속가능경영보고서를           │
│   │                                                  │      │ 발간합니다. 작성 기준 본 보고서는 지속가능경영    │
│   │                                                  │      │ 보고 기준인 GRI(G…                                │
│ 4 │ Samsung_Electronics_Sustainability_Report_2025_… │   84 │ 삼성전자 지속가능경영보고서 2025 84 코드 공시     │
│   │                                                  │      │ 항목 참고 페이지 및 답변 제품 안전 TC-HW-230a.1   │
│   │                                                  │      │ 제품의 정보보안 관련 리스크를 발견하고 이를       │
│   │                                                  │      │ 개선하는 방법 설명 삼성전자는 개인정보보호팀장,   │
│   │                                                  │      │ 정보보호센터장이 개인정보보호 운영위원회와 보안…  │
│ 5 │ Samsung_Electronics_Sustainability_Report_2025_… │   78 │ 삼성전자 지속가능경영보고서 2025 78 Scope 3       │
│   │                                                  │      │ 온실가스 배출량 검증 의견서 Our Company           │
│   │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People    │
└───┴──────────────────────────────────────────────────┴──────┴───────────────────────────────────────────────────┘

### 3. 리랭커

In [35]:
# uv add sentence-transformers
# uv add langchain_huggingface

- 우선 검색기로 후보군을 추출(10~30)
- 개수가 적거나 글자 수가 적으면 rerank를 해야할 이유가 없다.

In [47]:
ret_similarity = load_vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k" : 20}
)
question = "삼성과 현대 중에서 어느 기업이 더 나아"
ret_similarity_result = ret_similarity.invoke(question)
rich_docs(ret_similarity_result, title = "유사도 결과")

                                                    유사도 결과                                                    
┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃  # ┃ Source                                           ┃ Page ┃ Preview                                          ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  1 │ Samsung_Electronics_Sustainability_Report_2025_… │    5 │ 삼성전자 지속가능경영보고서 2025 05 Our Company  │
│    │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People   │
│    │                                                  │      │ 회사소개 About Us 삼성전자주식회사(이하          │
│    │                                                  │      │ 삼성전자)는 인재와 기술을 기반으로 최고의 제품 … │
│    │                                                  │      │ 서비스를…                                        │
│  2 │ Samsung_Electronics_Sustainability_Report_2025_… │    3 │ 삼성전자 지속가능경영보고서 2025 03 Our Company  │
│    │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People   │
│    │                                                  │      │ Our Company CEO 메시지 회사소개 기업 지배구조    │
│    │                                                  │      │ 중대성 평가 이해관계자 소통 04 05 06 …           │
│  3 │ Samsung_Electronics_Sustainability_Report_2025_… │   54 │ 삼성전자 지속가능경영보고서 2025 54 Bespoke AI   │
│    │                                                  │      │ 로봇청소기 보안 인증 획득 2024년에는 Bespoke AI  │
│    │                                                  │      │ 스팀 로봇청소기가 KISA 개인정보보호중심설계      │
│    │                                                  │      │ (PbD, Privacy by Design) 인증과 KISA IoT 보안    │
│    │                                                  │      │ 인증 중 …                                        │
│  4 │ Samsung_Electronics_Sustainability_Report_2025_… │   50 │ 삼성전자 지속가능경영보고서 2025 50 안전보건     │
│    │                                                  │      │ 삼성전자는 DX부문의 Global EHS실장, DS부문의     │
│    │                                                  │      │ 글로벌 제조&인프라  총괄장인 CSO(Chief Safety    │
│    │                                                  │      │ Officer)를 중심으로 2030년까지 상주  협력회사    │
│    │                                                  │      │ 안전보건 역량과 관리체계 …                       │
│  5 │ Samsung_Electronics_Sustainability_Report_2025_… │    4 │ 삼성전자 지속가능경영보고서 2025 04 Our Company  │
│    │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People   │
│    │                                                  │      │ 주주, 고객, 협력회사, 그리고 임직원 여러분,      │
│    │                                                  │      │ 2024년은 글로벌 지정학적 리스크와 AI 기술의 성 … │
│    │                                                  │      │ …                                                │
│  6 │ Samsung_Electronics_Sustainability_Report_2025_… │   61 │ 삼성전자 지속가능경영보고서 2025 61 Facts &      │
│    │                                                  │      │ Figures  경제성과 사회성과 환경성과   사업부문 … │
│    │                                                  │      │ 환경성과  62 63 68 72 Our Company AppendixFacts  │
│    │                                                  │      │ & Figures PrinciplePlanet People                 │
│  7 │ Samsung_Electronics_Sustainability_Report_2025_… │   45 │ 삼성전자 지속가능경영보고서 2025 45 협력회사     │
│    │                                                  │      │ 행동규범 삼성전자는 제품과 서비스를 제공하는     │
│    │                                                  │      │ 모든 협력회사에 노동인권, 환경, 안전보건, 윤리   │
│    │                                                  │      │ 분야를 포함하는 현지 법규와    RBA(Responsible   │
│    │                                                  │      │ Business Alliance) 1)  행…                       │
│  8 │ Samsung_Electronics_Sustainability_Report_2025_… │   40 │ 스타일에 맞

In [42]:
from langchain_community.cross_encoders.huggingface import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker

In [43]:
hf_ce = HuggingFaceCrossEncoder(
    model_name = "cross-encoder/ms-marco-MiniLM-L6-v2",
    model_kwargs = {
        "device" : "cuda",
        "max_length" : 512,
    }
)

In [48]:
compressor = CrossEncoderReranker(
    model = hf_ce,
    top_n = 20
)

reranker_retriever = ContextualCompressionRetriever(
    base_retriever = ret_similarity,
    base_compressor = compressor
)
reranker_result = reranker_retriever.invoke(question)
rich_docs(reranker_result, title = "리랭크 결과")

                                                    리랭크 결과                                                    
┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃  # ┃ Source                                           ┃ Page ┃ Preview                                          ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  1 │ Samsung_Electronics_Sustainability_Report_2025_… │   46 │ 정기적으로 모니터링하고 시스템으로 관리합니다.   │
│    │                                                  │      │ 하위 공급망 관리 지속적으로 증가하는 공급망      │
│    │                                                  │      │ 변동성과 불 확실성에 대응하기 위하여 삼성전자는  │
│    │                                                  │      │ 1 차 협력회사뿐 아니라 주요  품목을 공급하는     │
│    │                                                  │      │ 하위 협력회사에 대한 정보 관리를 점진적으로      │
│    │                                                  │      │ 확대하고 있습니다.…                              │
│  2 │ Samsung_Electronics_Sustainability_Report_2025_… │   45 │ 삼성전자 지속가능경영보고서 2025 45 협력회사     │
│    │                                                  │      │ 행동규범 삼성전자는 제품과 서비스를 제공하는     │
│    │                                                  │      │ 모든 협력회사에 노동인권, 환경, 안전보건, 윤리   │
│    │                                                  │      │ 분야를 포함하는 현지 법규와    RBA(Responsible   │
│    │                                                  │      │ Business Alliance) 1)  행…                       │
│  3 │ Samsung_Electronics_Sustainability_Report_2025_… │   45 │ 협력회사를 지원하고 교육합니다.  협력회사의      │
│    │                                                  │      │ 지속가능경영 관리 수준 향상을 위한 실사와 개     │
│    │                                                  │      │ 선조치 이행, 정보공개 등의 성과는 해 당 부서의   │
│    │                                                  │      │ 경영진과 실무자들의 KPI로 반영되며 목표 달성은   │
│    │                                                  │      │ 급여, 인센티브 등 보상과 연계됩니다. 추진 방향   │
│    │                                                  │      │ 삼성전자…                                        │
│  4 │ Samsung_Electronics_Sustainability_Report_2025_… │   54 │ 필수적으로 관리해야할 대상으로 인식합니다.       │
│    │                                                  │      │ 삼성전자 반도체의 주요  기술은 산업기술보호법 1… │
│    │                                                  │      │ 과 첨단전략산업법 2) 에 따라 보호되며, 매년 정 … │
│    │                                                  │      │ 감사로 보안 시스템의 유효성을 점검하여           │
│    │                                                  │      │ 보안사고를 예방합니다. 또한  국가핵심기술 보안   │
│    │                                                  │      │ 관리 지침을 …                                    │
│  5 │ Samsung_Electronics_Sustainability_Report_2025_… │   40 │ 스타일에 맞춰 혜택을 받을 수 있는 선택적         │
│    │                                                  │      │ 복리후생 제도를 운영합니다. ‧ 이사회 산하        │
│    │                                                  │      │ 노사관계 자문그룹 설치 ‧ 노동조합 공동교섭단과   │
│    │                                                  │      │ 교섭, 첫 단체 협약 체결(한국) ‧ 임금, 휴일 등을  │
│    │                                                  │      │ 포괄하는 임금협약 체결(한국) ‧  임금·단체협약    │
│    │                                                  │      │ 관련 잠정…                                       │
│  6 │ Samsung_Electronics_Sustainability_Report_2025_… │    5 │ 삼성전자 지속가능경영보고서 2025 05 Our Company  │
│    │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People   │
│    │                                                  │      │ 회사소개 About Us 삼성전자주식회사(이하          │
│    │                                                  │      │ 삼성전자)는 인재와 기술을 기반으로 최고의 제품 … │
│    │                                                  │      │ 서비스를…                                        │
│  7 │ Samsung_Electronics_Sustainability_Report_2025_… │   50 │ 반영하여 구직자 채 용을 지원하고, 삼성 협력회사  │
│    │            

### 4. 리오더
- 리랭크와 같이 사용
- 리랭크에서는 맥락(내용의 흐름) 고려 X

In [49]:
from langchain_community.document_transformers import LongContextReorder

reorder = LongContextReorder()
reorderded_result = reorder.transform_documents(reranker_result)

rich_docs(reorderded_result, title = "리오더 결과")

                                                    리오더 결과                                                    
┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃  # ┃ Source                                           ┃ Page ┃ Preview                                          ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  1 │ Samsung_Electronics_Sustainability_Report_2025_… │   45 │ 삼성전자 지속가능경영보고서 2025 45 협력회사     │
│    │                                                  │      │ 행동규범 삼성전자는 제품과 서비스를 제공하는     │
│    │                                                  │      │ 모든 협력회사에 노동인권, 환경, 안전보건, 윤리   │
│    │                                                  │      │ 분야를 포함하는 현지 법규와    RBA(Responsible   │
│    │                                                  │      │ Business Alliance) 1)  행…                       │
│  2 │ Samsung_Electronics_Sustainability_Report_2025_… │   54 │ 필수적으로 관리해야할 대상으로 인식합니다.       │
│    │                                                  │      │ 삼성전자 반도체의 주요  기술은 산업기술보호법 1… │
│    │                                                  │      │ 과 첨단전략산업법 2) 에 따라 보호되며, 매년 정 … │
│    │                                                  │      │ 감사로 보안 시스템의 유효성을 점검하여           │
│    │                                                  │      │ 보안사고를 예방합니다. 또한  국가핵심기술 보안   │
│    │                                                  │      │ 관리 지침을 …                                    │
│  3 │ Samsung_Electronics_Sustainability_Report_2025_… │    5 │ 삼성전자 지속가능경영보고서 2025 05 Our Company  │
│    │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People   │
│    │                                                  │      │ 회사소개 About Us 삼성전자주식회사(이하          │
│    │                                                  │      │ 삼성전자)는 인재와 기술을 기반으로 최고의 제품 … │
│    │                                                  │      │ 서비스를…                                        │
│  4 │ Samsung_Electronics_Sustainability_Report_2025_… │   54 │ 보호하여 온디바이스(On-Device)에 안전하게        │
│    │                                                  │      │ 저장됩니다. 2025년부터 녹스 볼트가 모바일        │
│    │                                                  │      │ 제품군뿐만 아니라 Bespoke AI 패밀리허브 냉장고,… │
│    │                                                  │      │ Bespoke AI 콤보 등의 스크린 탑재 스마트          │
│    │                                                  │      │ 가전제품에도 적용되어 사용 자가 쉽고 안전하게…   │
│  5 │ Samsung_Electronics_Sustainability_Report_2025_… │   86 │ 비록 삼성전자주식회사는 지속가능경영보고서의     │
│    │                                                  │      │ 미래 예측 진술이 시의성 있고 합리적인 정보, 가 … │
│    │                                                  │      │ 및 믿음에 기반한다고 판단하지만, 이러한 미래     │
│    │                                                  │      │ 예측 진술(그리고 이를 이루는 정보, 가정 및       │
│    │                                                  │      │ 믿음)은  다양한 요인, 리스크, 불확실성의         │
│    │                                                  │      │ 영향권에 있으므로…                               │
│  6 │ Samsung_Electronics_Sustainability_Report_2025_… │   50 │ 삼성전자 지속가능경영보고서 2025 50 안전보건     │
│    │                                                  │      │ 삼성전자는 DX부문의 Global EHS실장, DS부문의     │
│    │                                                  │      │ 글로벌 제조&인프라  총괄장인 CSO(Chief Safety    │
│    │                                                  │      │ Officer)를 중심으로 2030년까지 상주  협력회사    │
│    │                                                  │      │ 안전보건 역량과 관리체계 …                       │
│  7 │ Samsung_Electronics_Sustainability_Report_2025_… │   43 │ 삼성전자 지속가능경영보고서 2025 43 추진 체계    │
│    │                                                  │      │ 삼성전자는 부문별 최고안전보건책임자(CSO, Chief  │
│    │                                                  │      │ Safety Officer) 를 두고 DX부문은 Glo